## Importing the modules

In [1]:
import librosa.display
import numpy as np
import random
import audiomentations as A
import soundfile as sf
import os
import librosa
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Data augmentation, Data pre-processing & Data Normalization

In [2]:
def preprocess_audio(audio_folder, num_mfcc_coefficients, desired_shape):
    preprocessed_data = []
    preprocessed_labels = []
    
    for index, cls in enumerate(os.listdir(audio_folder)):
        class_folder = os.path.join(audio_folder, cls)
        for file in os.listdir(class_folder)[:15]:
            audio_file = os.path.join(class_folder, file)
            original_audio, sr = sf.read(audio_file)

            # Apply augmentation to create augmented audio
            augment1 = A.AddGaussianNoise(p=0.2)
            augment2 = A.TimeStretch(p=0.2)
            augment3 = A.PitchShift(p=0.2)
            augment4 = A.Shift(p=0.2)
            augment5 = A.TimeMask(p=0.2)

            augmented_audio1 = augment1(samples=original_audio, sample_rate=sr)
            augmented_audio2 = augment2(samples=original_audio, sample_rate=sr)
            augmented_audio3 = augment3(samples=original_audio, sample_rate=sr)
            augmented_audio4 = augment4(samples=original_audio, sample_rate=sr)
            augmented_audio5 = augment5(samples=original_audio, sample_rate=sr)

            # Perform feature extraction (e.g., MFCCs) on original and augmented audio
            for audio in [original_audio, augmented_audio1, augmented_audio2, augmented_audio3, augmented_audio4, augmented_audio5]:
                mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=num_mfcc_coefficients)

                # Normalize the MFCCs (optional but recommended)
                mfccs = (mfccs - np.mean(mfccs)) / np.std(mfccs)

                # Reshape or pad the MFCCs to match the desired input shape
                num_frames = mfccs.shape[1]
                if num_frames < desired_shape[0]:
                    mfccs = np.pad(mfccs, ((0, 0), (0, desired_shape[0] - num_frames)), mode='constant')
                elif num_frames > desired_shape[0]:
                    mfccs = mfccs[:, :desired_shape[0]]

                # Append the preprocessed data and label
                preprocessed_data.append(mfccs.T)  # Transpose the data
                preprocessed_labels.append(index)

    # Stack the preprocessed data into a 3D array
    X = np.array(preprocessed_data).reshape(-1,13)
    y = np.array(preprocessed_labels)


    return X, y


## Define the data folder path

In [18]:
# Define your audio folder path
audio_folder = "donateacry"

# Function to remove ".DS_Store" files
def remove_ds_store_files(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file == ".DS_Store":
                ds_store_file_path = os.path.join(root, file)
                os.remove(ds_store_file_path)

# Remove ".DS_Store" files from the audio folder
remove_ds_store_files(audio_folder)

# Now, you can proceed with your audio preprocessing as before
num_mfcc_coefficients = 13
desired_shape = (1, 13)  # Adjust the desired shape as needed
X, y = preprocess_audio(audio_folder, num_mfcc_coefficients, desired_shape)

In [17]:
print("Shape of audio files:",X.shape)
print("Shape of labels:",y.shape)
X

Shape of audio files: (6, 13)
Shape of labels: (6,)


array([[-8.20595525e+00,  1.51498114e+00,  2.46539512e-01,
         2.50964196e-01,  8.23222623e-03,  1.01491315e-02,
         1.43629409e-01, -5.06586207e-03,  6.29581766e-02,
         6.96103339e-02,  9.25116507e-02,  3.60205983e-01,
         3.23634069e-01],
       [-8.20595455e+00,  1.51498127e+00,  2.46539488e-01,
         2.50964195e-01,  8.23222287e-03,  1.01490775e-02,
         1.43629432e-01, -5.06582251e-03,  6.29582033e-02,
         6.96102753e-02,  9.25116539e-02,  3.60206008e-01,
         3.23634058e-01],
       [-5.38009453e+00,  1.14895034e+00,  1.36143550e-01,
         2.62174338e-01,  5.64717576e-02,  5.63273132e-02,
         1.87074363e-01,  5.21514863e-02,  1.09254755e-01,
         1.26343042e-01,  9.56935063e-02,  3.42191845e-01,
         2.15401724e-01],
       [-8.20595455e+00,  1.51498127e+00,  2.46539488e-01,
         2.50964195e-01,  8.23222287e-03,  1.01490775e-02,
         1.43629432e-01, -5.06582251e-03,  6.29582033e-02,
         6.96102753e-02,  9.25116539e

## Data sampling

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=42)

## Model training

## Decision tree

In [6]:
dt = DecisionTreeClassifier(max_depth=5)
dt.fit(X_train,y_train)
dt_pred = dt.predict(X_test)
print("Accuracy score of Decision tree:",accuracy_score(y_test,dt_pred))

Accuracy score of Decision tree: 0.7560975609756098


## Random forest classifier

In [7]:
rfc = RandomForestClassifier(random_state=32)
rfc.fit(X_train,y_train)
rfc_pred = rfc.predict(X_test)
print("Accuracy score of Random Forest:",accuracy_score(y_test,rfc_pred))

Accuracy score of Random Forest: 0.9512195121951219


## Gradient Boosting

In [8]:
gb = GradientBoostingClassifier()
gb.fit(X_train,y_train)
gb_pred = gb.predict(X_test)
print("Accuracy score of Gradient Boosting:",accuracy_score(y_test,gb_pred))

Accuracy score of Gradient Boosting: 0.9512195121951219


## Support vector machine

In [9]:
svm = SVC()
svm.fit(X_train,y_train)
svm_pred = svm.predict(X_test)
print("Accuracy score of SVC:",accuracy_score(y_test,svm_pred))

Accuracy score of SVC: 0.3902439024390244


In [ ]:
Ap = "donateacry\belly_pain\69BDA5D6-0276-4462-9BF7-951799563728-1436936185-1.1-m-26-bp.wav"
